<a href="https://colab.research.google.com/github/JuanVCPeru/Taller/blob/main/Pregunta_3_Examen_Parcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import pipeline
import os

# Configuración de tqdm para su uso en pandas
tqdm.pandas()

# Función para limpiar el texto del abstract
def clean_text(text):
    if pd.isna(text) or text.strip() == '':
        return ''
    return text

# Función para resumir el texto del abstract
def summarize_text(text, summarizer):
    if text == '':
        return ''
    try:
        return summarizer(text, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    except Exception as e:
        print(f"Error al resumir el texto: {e}")
        return ''

# Leer el archivo CSV
file_path = '/content/ResearchRabbit_Export_1725123890.csv'  # Actualiza este camino según sea necesario
df = pd.read_csv(file_path)

# Asegúrate de que la columna 'Abstract' esté presente
if 'Abstract' not in df.columns:
    raise ValueError("La columna 'Abstract' no se encuentra en el archivo CSV")

# Cargar el modelo de resumen y el modelo de pregunta-respuesta
summarizer = pipeline("summarization")
qa_pipeline = pipeline("question-answering")

# Limpiar y resumir los abstracts con una barra de progreso
df['Cleaned_Abstract'] = df['Abstract'].progress_apply(clean_text)
df['Summary'] = df['Cleaned_Abstract'].progress_apply(lambda x: summarize_text(x, summarizer))

# Guardar el DataFrame procesado en un archivo CSV en el directorio de Colab
output_path_colab = '/content/bibliografia_resumida.csv'
df.to_csv(output_path_colab, index=False)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Error al resumir el texto: index out of range in self


 97%|█████████▋| 980/1006 [1:36:45<01:59,  4.58s/it]

Error al resumir el texto: index out of range in self


100%|██████████| 1006/1006 [1:38:58<00:00,  5.90s/it]


OSError: Cannot save file into a non-existent directory: '/path/to/your/local/directory'

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import pipeline
import os

# Configuración de tqdm para su uso en pandas
tqdm.pandas()

# Leer el archivo CSV
file_path = '/content/bibliografia_resumida.csv'  # Actualiza este camino según sea necesario
df = pd.read_csv(file_path)

# Cargar el modelo de pregunta-respuesta  <-- Add this line to define qa_pipeline
qa_pipeline = pipeline("question-answering")

# Función para responder preguntas basadas en los abstracts
def answer_question(question, df, qa_pipeline):
    # Concatenar todos los abstracts en un solo texto para que el modelo pueda usar todo el contexto
    context = ' '.join(df['Cleaned_Abstract'].dropna())
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Interacción con el usuario
def user_interaction():
    while True:
        print("\nOpciones:")
        print("1. Realizar una pregunta sobre los abstracts")
        print("2. Ver ejemplos de abstracts")
        print("3. Ver estadísticas de los abstracts")
        print("4. Guardar datos procesados")
        print("5. Salir")

        choice = input("Elige una opción (1-5): ").strip()

        if choice == '1':
            question = input("¿Qué pregunta quieres hacer sobre los abstracts? ")
            answer = answer_question(question, df, qa_pipeline)
            print(f"Respuesta: {answer}")
        elif choice == '2':
            num_examples = int(input("¿Cuántos ejemplos de abstracts deseas ver? "))
            print("\nEjemplos de abstracts:")
            for i in range(min(num_examples, len(df))):
                print(f"\nAbstract {i+1}:")
                print(df.iloc[i]['Abstract'])
        elif choice == '3':
            total_abstracts = len(df)
            non_empty_abstracts = df['Abstract'].notna().sum()
            print(f"\nTotal de abstracts: {total_abstracts}")
            print(f"Abstracts no vacíos: {non_empty_abstracts}")
            print(f"Abstracts vacíos o nulos: {total_abstracts - non_empty_abstracts}")
        elif choice == '4':
            df.to_csv(output_path_colab, index=False)
            df.to_csv(output_path_local, index=False)
            print(f"Datos guardados en:\nColab: {output_path_colab}\nLocal: {output_path_local}")
        elif choice == '5':
            print("¡Hasta luego!")
            break
        else:
            print("Opción no válida, por favor elige entre 1 y 5.")

# Iniciar la interacción con el usuario
user_interaction()

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



Opciones:
1. Realizar una pregunta sobre los abstracts
2. Ver ejemplos de abstracts
3. Ver estadísticas de los abstracts
4. Guardar datos procesados
5. Salir
Elige una opción (1-5): 3

Total de abstracts: 1006
Abstracts no vacíos: 748
Abstracts vacíos o nulos: 258

Opciones:
1. Realizar una pregunta sobre los abstracts
2. Ver ejemplos de abstracts
3. Ver estadísticas de los abstracts
4. Guardar datos procesados
5. Salir
Elige una opción (1-5): 1
¿Qué pregunta quieres hacer sobre los abstracts? Que abstract tiene los porcentajes mas altos de exito en la prueba
Respuesta: ISABEL BORRÁS

Opciones:
1. Realizar una pregunta sobre los abstracts
2. Ver ejemplos de abstracts
3. Ver estadísticas de los abstracts
4. Guardar datos procesados
5. Salir
Elige una opción (1-5): 1
¿Qué pregunta quieres hacer sobre los abstracts? Que metodo de aprendizaje es el mas utilizado
Respuesta: El objetivo de este trabajo

Opciones:
1. Realizar una pregunta sobre los abstracts
2. Ver ejemplos de abstracts
3. V

KeyboardInterrupt: Interrupted by user